<a href="https://colab.research.google.com/github/amanmaurya7/langchain_collab/blob/main/Research_Paper_Summarizer_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install -U langchain-community
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.6/330.6 kB 11.7 MB/s eta 0:00:00


In [4]:
import os
os.environ["PPLX_API_KEY"] = "" # your perplexity api key here

In [16]:
from langchain_community.chat_models import ChatPerplexity
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnableSequence
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

llm = ChatPerplexity(model = 'sonar-pro', temperature=0.3, max_tokens = 500)

In [17]:
loader = PyPDFLoader('research_paper.pdf') # load the pdf to document loaders
docs = loader.load()

In [32]:
#added a textsplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap = 10,
    separators = ['\n\n','\n','.',' ']
)

texts = text_splitter.split_documents(docs)

In [33]:
# A prompt template for the Summary
template = """
You are a expert assistant for Research Paper Summarizer
###Documents
{chunk_document}

###Instructions
Extract a structured summary of this three sections:
1.Problem Statement
2.Proposed Method
3.Key Findings

Return the summary in JSON Format with keys: "Problem Statement", "Proposed Method", "Key Findings"
"""

In [34]:
prompt = PromptTemplate.from_template(
    template = template
)

In [35]:
chain = prompt | llm | StrOutputParser()

In [39]:
#getting the first 3 docs summary

for i, doc in enumerate(texts[:3]):
  summary = chain.invoke({'chunk_document':doc.page_content})
  print(f'\n --- Summary{i+1} -- \n{summary} ')


 --- Summary1 -- 
{
  "Problem Statement": "Current AI agent benchmarks and evaluation practices have several shortcomings that hinder real-world usefulness: (1) narrow focus on accuracy without considering cost, leading to needlessly complex and costly state-of-the-art (SOTA) agents and mistaken conclusions about accuracy gains; (2) conflation of benchmarking needs for model developers and downstream application developers, making it hard to select suitable agents; (3) inadequate or missing holdout sets, causing overfitting and fragile agents that take shortcuts; (4) lack of standardization, leading to poor reproducibility.[1][2][5]",
  "Proposed Method": "The authors propose jointly optimizing accuracy and cost, visualizing results as Pareto curves to guide agent design; introduce three simple baseline agents that leverage stochasticity of underlying models (e.g., multiple calls) to achieve high performance at lower cost; advocate for cost-controlled evaluations, better holdout sets